# RadTract example pipeline

This notebook will show you how to use RadTract to calculate radiomics tractometry features. The used data can be found in resources/example/ in this repository. Make sure to install RadTract before running this notebook: `pip install radtract`.

To run RadTract on a single subject, you need at least the following items:

1. A tractogram, i.e., a collection of streamlines, for the tract you are interested in (here `resources/example/CST_right.trk`). 
2. The image you want to calculate features from, for example a fractional anisotropy (FA) map (here `resources/example/fa.nii.gz`).

The code below is the tl;dr version of how to use these items to calculate your tract-specific features. Keep in mind to read the longer example below before using RadTract for a multi-subject study!

In [ ]:
# parcellate tract
!radtract_parcellate --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz --output resources/example/CST_right_parcellation.nii.gz

# claculate features using this parcellation
!radtract_features --parcellation resources/example/CST_right_parcellation.nii.gz --map resources/example/fa.nii.gz --output resources/example/CST_right_features.csv

## Tract Parcellation

The above example uses default parameters and might not be optimal for your usecase. For example, the automatically estimated number of parcels might vary between subjects, thus rendering them incompatible for a future joint statistical analysis or machine learning experiment. To avoid this, it is possible to manually set the number of parcel (`--num_parcels`). 

Also, when processing multiple subjects, it is recommended to define the start of the tract using a binary segmentation (`--start`). If this is not done, it might happen that the same tract in a different subject might be reversed.

RadTract further offers multiple types (`--type`) of parcellations, namely hyperplane (default), centerline and static and the parcellation can happen in voxel-space (default) or in streamline-space (classic tractometry, option `--streamline_space True`). 

Here you go with a full list of the available command line options (more available via the python interface):

In [ ]:
!radtract_parcellate

In Figure 1, the different types of parcellations are illustrated. Below you will find the corresponding commands used to create these results.

![](resources/parcellations_overview.png)

Figure 1: Hyperplane-based parcellations of the right CST are visualized in voxel- (a) and streamline-space (c). (b) and (d) show the corresponding centerline-based parcellations. The static parcellation is only vaid in streamline-space, since it parcellates each streamline individually (e).

In [18]:
# # Figure 1 (a)
!radtract_parcellate --type hyperplane --output resources/example/CST_left_parcellation_hyperplane.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

# # Figure 1 (b)
!radtract_parcellate --type centerline --output resources/example/CST_left_parcellation_centerline.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

# # Figure 1 (c)
!radtract_parcellate --type hyperplane --streamline_space --output resources/example/CST_left_streamline-parcellation_hyperplane.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

# # Figure 1 (d)
!radtract_parcellate --type centerline --streamline_space --output resources/example/CST_left_streamline-parcellation_centerline.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

# Figure 1 (e)
!radtract_parcellate --type static --output resources/example/CST_left_streamline-parcellation_static.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

## Feature calculation

The actual feature calculation is pretty straight forward and the tl;dr example above works perfectly well for FA maps. For other maps, e.g., ADC, it might be neccessary to adjust the pyradiomics parameter file, particularly the parameter `binWidt`. Check out the available .yaml parameter files in `radtract/`. For details, please check out the pyradiomics documentation https://pyradiomics.readthedocs.io/en/latest/ 

In [17]:
!radtract_features


usage: radtract_features [-h] [--parcellation PARCELLATION] [--map MAP]
                         [--pyrad_params PYRAD_PARAMS] [--output OUTPUT]

RadTract Feature Calculation

options:
  -h, --help            show this help message and exit
  --parcellation PARCELLATION
                        Input parcellation file
  --map MAP             Parameter map file (e.g. fractional anisotropy)
  --pyrad_params PYRAD_PARAMS
                        Pyradiomics parameter file (e.g. pyrad.yaml)
  --output OUTPUT       Output feature file (.csv)


## How to get the tracts/streamlines and start-region-images in the first place

You can obtain tractograms/streamlines of individual tracts with various software tools, e.g. MRtrix or MITK Diffusion. Since it is easy to use and fully automatic, we recommend TractSeg (https://github.com/MIC-DKFZ/TractSeg/) for larger studies. Simply install TractSeg via `pip install tractseg` and make sure pytorch is also installed (`pip install torch`).

In [ ]:
# segment 72 tracts in the input dwi
!TractSeg -i resources/example/dwi.nii.gz --raw_diffusion_input --keep_intermediate_files

# segment start- and end-regions of all tracts. these are required for the following tractography as well as for the parcellation
!TractSeg -i resources/example/tractseg_output/peaks.nii.gz --output_type endings_segmentation

# create tract orientation maps (TOM) for tractography 
!TractSeg -i resources/example/tractseg_output/peaks.nii.gz --output_type TOM

# run tractography of all tracts. we use more streamlines (10000) than the defaul to obtain a better tract coverage.
!Tracking -i resources/example/tractseg_output/peaks.nii.gz --tracking_format trk -nr_fibers 10000